In [1]:
# imports
%run SMC_ABC_recoded.py
%matplotlib inline
import DataAnalysisFunctions as daf

%px import numpy as np
%px import scipy as sp
%px import random
%px from scipy import stats
%px from ipyparallel import CompositeError

rc[:]['create_parameter_vector'] = create_parameter_vector
rc[:]['singleTranscript'] = singleTranscript

# load experimental data
Data0pM = np.load('Data0pM.npy')
Data5pM = np.load('Data5pM.npy')
Data7pM = np.load('Data7pM.npy')
Data10pM = np.load('Data10pM.npy')
Data14pM = np.load('Data14pM.npy')
Data20pM = np.load('Data20pM.npy')
Data100pM = np.load('Data100pM.npy')
Data1000pM = np.load('Data1000pM.npy')

Mock0pM = np.load('Mock0pM.npy')
Mock5pM = np.load('Mock5pM.npy')
Mock7pM = np.load('Mock7pM.npy')
Mock10pM = np.load('Mock10pM.npy')
Mock14pM = np.load('Mock14pM.npy')
Mock20pM = np.load('Mock20pM.npy')
Mock100pM = np.load('Mock100pM.npy')
Mock1000pM = np.load('Mock1000pM.npy')

/Users/stephan/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Using Sequential Monte Carlo Approximate Bayesian Computation to fit experimental data

# Create a large start population by random sampling

Generate about 50000 candidate particles sampled from prior distributions. Simulate mRNA count only and save to file. Make the number of simulated cells larger than actual data sets. Before running SMC ABC take the start population, add noise and then find the best candidate particles for the SMC fitting. Reuse the start population for each fit be it real data or benchmark data. This should increase the sampling of the model and parameter space substantially.

The resulting arrays of the simulations can become very big, thus it is easier not to hold all 50000 particles into one file but save 2000 or so particles into one file.

In [2]:
# parameters

# population size for SMC ABC
n_particles = 2000

# stop distance
stop = 0.2

# maximal number of iterations
iterations = 1

# path to save the results
path = '/Users/stephan/Desktop/gene_transcription_SMC_ABC/fits/'

# folder to temporally store individual particles to find a good start population
temp = '/Users/stephan/Desktop/temp/'

# path to candidate particles from the large start population
cand_path = '/Users/stephan/Desktop/gene_transcription_SMC_ABC/cand/'

In [10]:
# number of files to create
rounds = 2
start = 0
count = start
# sampled candidate particles per file
n_particles = 50
n_sim = 100
while count - start < rounds:
    print count - start
    res = smc_start_multi(n_particles,n_sim)
    np.save(cand_path+'res_'+str(count)+'.npy',res)
    count = count + 1

0
Duration: 3.13
1
Duration: 3.14


# Fit experimental data from estrogen dose response

### 0pM, 2nd

In [29]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_0pM,par_cand_0pM = particle_distance_lut(Data0pM,Mock0pM,paths)

# save distances and parameters of all candidate particles
np.save(path + '0pM/dist_cand_10pM.npy',dist_cand_0pM)
np.save(path+'0pM/par_cand_10pM.npy',par_cand_0pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_0pM,n_particles)
np.save(path+'0pM/start_0pM_2000_corrected.npy',start_population)

File: 1 / 2
File: 2 / 2
Duration: 0.34 min
Duration: 0.01 min


In [3]:
start_population = np.load(path+'0pM/start_0pM_2000_corrected.npy')
start_population[0][:,0] = 1./start_population[0].shape[0]

In [4]:
alpha = 33.15
threshold = 0.6
save = [True,path+'0pM/','smc_0pM_']
smc = smc_abc(Data0pM,Mock0pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'0pM/smc_0pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.811
Mean start distance: 3.313
Max start distance: 4.83
STD of start distance: 1.014
Mean of burst, tau and T: [ 64.449   9.297  63.678]
STD of burst, tau and T: [ 106.679    9.611   67.646]
Iteration: 1
Threshold distance 2.653


<string>:838: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
<string>:839: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Try: 0 First particles accepted: 71
Try: 1 More particles accepted: 85
Enough particles accepted: 85
# Created particles: 98
# particles better than stop: 0
Min distance of current population: 0.531
Mean distance of current population: 1.631
Max distance of current population: 2.57
Mean of burst, tau and T: [   5.598    2.194  173.988]
STD of burst, tau and T: [   6.104    2.147  190.286]
Duration of 1 th iteration 0.81 min
Last iteration finished, max distance: 2.56965204996
Duration of iterations: 0.81 min


#### Fit 115 model to 0 pM 

The 115 model (two state with combined source of extrinsic noise of RNA pol II elongation rate and initiation rate) is by far the most frequent model for the low estrogen concentrations. Fitting this model with fixed topology allows reduce the dimensionality of the search space. The 115 model has four kinetic parameters. Fitting all data sets with the same model topology is a way to assess the dependency of the model parameters on experimental conditions.

In [12]:
# find the best 1000 particles in the posterior distribution for the 0 pM data set
ind = np.where(smc_0pM[0][:,1,-1] == 5)[0]
ind = ind[0:1000]
start_population = [smc_0pM[0][ind,:,-1],smc_0pM[1][:,:,ind],smc_0pM[2][ind,:,-1],smc_0pM[3][:,:,ind]]
start_population[0][:,0] = 1./1000.

In [13]:
alpha = 33.15

# the threshold value defines the likelihood for changes on the model space. A value of one prohibits any jumps at 
# all and thus keeps the model topology fixed.
threshold = 1.
save = [True,path+'0pM/','smc_0pM_115_']
smc = smc_abc(Data0pM,Mock0pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'0pM/smc_0pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.504
Mean start distance: 0.61
Max start distance: 0.64
STD of start distance: 0.024
Mean of burst, tau and T: [   4.168    1.618  309.055]
STD of burst, tau and T: [  2.836   1.104  83.979]
Iteration: 1
Threshold distance 0.591
Try: 0 First particles accepted: 15
Try: 1 More particles accepted: 20
Try: 2 More particles accepted: 32
Try: 3 More particles accepted: 37
Try: 4 More particles accepted: 52
Try: 5 More particles accepted: 67
Try: 6 More particles accepted: 85
Try: 7 More particles accepted: 100
Try: 8 More particles accepted: 116
Try: 9 More particles accepted: 135
Try: 10 More particles accepted: 143
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 155
Try: 12 More particles accepted: 171
Try: 13 More particles accepted: 180
Try: 14 More particles accepted: 192
Not enough particles accepted: 192
# Created particles: 10692
# particles better than stop: 0
Min distance of curren

### 5pM, 2nd

In [4]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_5pM,par_cand_5pM = particle_distance_lut(Data5pM,Mock5pM,paths)
np.save(path + '5pM/dist_cand_10pM.npy',dist_cand_5pM)
np.save(path+'5pM/par_cand_10pM.npy',par_cand_5pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_5pM,n_particles)
np.save(path+'5pM/start_5pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 147.7 min
Duration: 2.64 min


In [5]:
start_population = np.load(path+'5pM/start_5pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [6]:
alpha = 33.15
threshold = 0.6
save = [True,path+'5pM/','smc_5pM_']
smc = smc_abc(Data5pM,Mock5pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'5pM/smc_5pM_1000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.477
Mean start distance: 0.877
Max start distance: 0.989
STD of start distance: 0.09
Mean of burst, tau and T: [   7.751    2.055  107.305]
STD of burst, tau and T: [  5.778   1.813  68.306]
Iteration: 1
Threshold distance 0.805


<string>:748: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
<string>:749: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Try: 0 First particles accepted: 4
Try: 1 More particles accepted: 34
Try: 2 More particles accepted: 80
Try: 3 More particles accepted: 128
Try: 4 More particles accepted: 168
Try: 5 More particles accepted: 216
Try: 6 More particles accepted: 268
Try: 7 More particles accepted: 306
Try: 8 More particles accepted: 356
Try: 9 More particles accepted: 406
Try: 10 More particles accepted: 452
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 482
Try: 12 More particles accepted: 518
Try: 13 More particles accepted: 548
Try: 14 More particles accepted: 578
Not enough particles accepted: 578
# Created particles: 20034
# particles better than stop: 0
Min distance of current population: 0.477
Mean distance of current population: 0.792
Max distance of current population: 0.845
Mean of burst, tau and T: [  3.814   0.983  54.197]
STD of burst, tau and T: [  2.986   0.728  36.353]
Duration of 1 th iteration 147.65 min
Iteration: 2
Threshold distance 0

#### Fit 115 model to 5 pM

In [14]:
ind = np.where(smc_5pM[0][:,1,-1] == 5)[0]
ind = ind[0:1000]
start_population = [smc_5pM[0][ind,:,-1],smc_5pM[1][:,:,ind],smc_5pM[2][ind,:,-1],smc_5pM[3][:,:,ind]]
start_population[0][:,0] = 1./1000.

In [15]:
alpha = 33.15
threshold = 1.
save = [True,path+'5pM/','smc_5pM_115_']
smc = smc_abc(Data5pM,Mock5pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'5pM/smc_5pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.507
Mean start distance: 0.622
Max start distance: 0.656
STD of start distance: 0.027
Mean of burst, tau and T: [  3.138   0.807  42.974]
STD of burst, tau and T: [  1.801   0.295  13.19 ]
Iteration: 1
Threshold distance 0.601
Try: 0 First particles accepted: 23
Try: 1 More particles accepted: 45
Try: 2 More particles accepted: 56
Try: 3 More particles accepted: 70
Try: 4 More particles accepted: 92
Try: 5 More particles accepted: 112
Try: 6 More particles accepted: 141
Try: 7 More particles accepted: 161
Try: 8 More particles accepted: 184
Try: 9 More particles accepted: 198
Try: 10 More particles accepted: 222
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 247
Try: 12 More particles accepted: 268
Try: 13 More particles accepted: 281
Try: 14 More particles accepted: 296
Not enough particles accepted: 296
# Created particles: 9900
# particles better than stop: 0
Min distance of curren

### 7 pM, 2nd

In [7]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_7pM,par_cand_7pM = particle_distance_lut(Data7pM,Mock7pM,paths)
np.save(path + '7pM/dist_cand_10pM.npy',dist_cand_7pM)
np.save(path+'7pM/par_cand_10pM.npy',par_cand_7pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_7pM,n_particles)
np.save(path+'7pM/start_7pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 132.65 min
Duration: 2.33 min


In [14]:
start_population = np.load(path+'7pM/start_7pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [15]:
alpha = 33.15
threshold = 0.6
save = [True,path+'7pM/','smc_7pM_']
smc = smc_abc(Data7pM,Mock7pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'7pM/smc_7pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.589
Mean start distance: 0.96
Max start distance: 1.054
STD of start distance: 0.081
Mean of burst, tau and T: [  7.404   1.679  63.139]
STD of burst, tau and T: [  5.13    1.265  41.921]
Iteration: 1
Threshold distance 0.897
Try: 0 First particles accepted: 12
Try: 1 More particles accepted: 63
Try: 2 More particles accepted: 112
Try: 3 More particles accepted: 167
Try: 4 More particles accepted: 224
Try: 5 More particles accepted: 264
Try: 6 More particles accepted: 319
Try: 7 More particles accepted: 377
Try: 8 More particles accepted: 423
Try: 9 More particles accepted: 466
Try: 10 More particles accepted: 500
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 535
Try: 12 More particles accepted: 579
Try: 13 More particles accepted: 639
Try: 14 More particles accepted: 682
Not enough particles accepted: 682
# Created particles: 19320
# particles better than stop: 0
Min distance of cur

#### Fit 115 model to 7 pM

In [16]:
ind = np.where(smc_7pM[0][:,1,-1] == 5)[0]
ind = ind[0:1000]
start_population = [smc_7pM[0][ind,:,-1],smc_7pM[1][:,:,ind],smc_7pM[2][ind,:,-1],smc_7pM[3][:,:,ind]]
start_population[0][:,0] = 1./1000.

In [17]:
alpha = 33.15
threshold = 1.
save = [True,path+'7pM/','smc_7pM_115_']
smc = smc_abc(Data7pM,Mock7pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'7pM/smc_7pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.522
Mean start distance: 0.648
Max start distance: 0.679
STD of start distance: 0.026
Mean of burst, tau and T: [  4.527   0.709  34.341]
STD of burst, tau and T: [ 2.539  0.243  8.123]
Iteration: 1
Threshold distance 0.628
Try: 0 First particles accepted: 8
Try: 1 More particles accepted: 19
Try: 2 More particles accepted: 35
Try: 3 More particles accepted: 44
Try: 4 More particles accepted: 52
Try: 5 More particles accepted: 60
Try: 6 More particles accepted: 71
Try: 7 More particles accepted: 82
Try: 8 More particles accepted: 94
Try: 9 More particles accepted: 103
Try: 10 More particles accepted: 109
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 117
Try: 12 More particles accepted: 125
Try: 13 More particles accepted: 130
Try: 14 More particles accepted: 142
Not enough particles accepted: 142
# Created particles: 10951
# particles better than stop: 0
Min distance of current popul

### 10 pM, 2nd

In [16]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_10pM,par_cand_10pM = particle_distance_lut(Data10pM,Mock10pM,paths)
np.save(path + '10pM/dist_cand_10pM.npy',dist_cand_10pM)
np.save(path+'10pM/par_cand_10pM.npy',par_cand_10pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_10pM,n_particles)
np.save(path+'10pM/start_10pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 138.76 min
Duration: 2.51 min


In [17]:
start_population = np.load(path+'10pM/start_10pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [18]:
alpha = 33.15
threshold = 0.6
save = [True,path+'10pM/','smc_10pM_']
smc = smc_abc(Data10pM,Mock10pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'10pM/smc_10pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.647
Mean start distance: 1.004
Max start distance: 1.115
STD of start distance: 0.092
Mean of burst, tau and T: [  9.159   1.864  49.1  ]
STD of burst, tau and T: [  5.997   1.369  32.529]
Iteration: 1
Threshold distance 0.93
Try: 0 First particles accepted: 8
Try: 1 More particles accepted: 47
Try: 2 More particles accepted: 94
Try: 3 More particles accepted: 132
Try: 4 More particles accepted: 180
Try: 5 More particles accepted: 205
Try: 6 More particles accepted: 252
Try: 7 More particles accepted: 292
Try: 8 More particles accepted: 336
Try: 9 More particles accepted: 380
Try: 10 More particles accepted: 418
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 458
Try: 12 More particles accepted: 491
Try: 13 More particles accepted: 522
Try: 14 More particles accepted: 560
Not enough particles accepted: 560
# Created particles: 20185
# particles better than stop: 0
Min distance of curre

#### Fit 115 model to 10 pM

In [18]:
ind = np.where(smc_10pM[0][:,1,-1] == 5)[0]
ind = ind[0:1000]
start_population = [smc_10pM[0][ind,:,-1],smc_10pM[1][:,:,ind],smc_10pM[2][ind,:,-1],smc_10pM[3][:,:,ind]]
start_population[0][:,0] = 1./1000.

In [19]:
alpha = 33.15
threshold = 1.
save = [True,path+'10pM/','smc_10pM_115_']
smc = smc_abc(Data10pM,Mock10pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'10pM/smc_10pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.541
Mean start distance: 0.708
Max start distance: 0.748
STD of start distance: 0.034
Mean of burst, tau and T: [  5.67    0.805  27.892]
STD of burst, tau and T: [ 3.262  0.269  7.599]
Iteration: 1
Threshold distance 0.681
Try: 0 First particles accepted: 23
Try: 1 More particles accepted: 33
Try: 2 More particles accepted: 44
Try: 3 More particles accepted: 55
Try: 4 More particles accepted: 66
Try: 5 More particles accepted: 79
Try: 6 More particles accepted: 94
Try: 7 More particles accepted: 105
Try: 8 More particles accepted: 112
Try: 9 More particles accepted: 123
Try: 10 More particles accepted: 132
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 146
Try: 12 More particles accepted: 151
Try: 13 More particles accepted: 162
Try: 14 More particles accepted: 169
Not enough particles accepted: 169
# Created particles: 10675
# particles better than stop: 0
Min distance of current po

### 14 pM

In [19]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_14pM,par_cand_14pM = particle_distance_lut(Data14pM,Mock14pM,paths)
np.save(path + '14pM/dist_cand_14pM.npy',dist_cand_14pM)
np.save(path+'14pM/par_cand_14pM.npy',par_cand_14pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_14pM,n_particles)
np.save(path+'14pM/start_14pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 127.94 min
Duration: 2.24 min


In [20]:
start_population = np.load(path+'14pM/start_14pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [21]:
alpha = 33.15
threshold = 0.6
save = [True,path+'14pM/','smc_14pM_']
smc = smc_abc(Data14pM,Mock14pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'14pM/smc_14pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.655
Mean start distance: 1.078
Max start distance: 1.191
STD of start distance: 0.095
Mean of burst, tau and T: [ 10.614   2.192  39.029]
STD of burst, tau and T: [  6.214   1.62   22.248]
Iteration: 1
Threshold distance 1.007
Try: 0 First particles accepted: 5
Try: 1 More particles accepted: 20
Try: 2 More particles accepted: 31
Try: 3 More particles accepted: 47
Try: 4 More particles accepted: 62
Try: 5 More particles accepted: 76
Try: 6 More particles accepted: 96
Try: 7 More particles accepted: 119
Try: 8 More particles accepted: 136
Try: 9 More particles accepted: 159
Try: 10 More particles accepted: 181
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 194
Try: 12 More particles accepted: 208
Try: 13 More particles accepted: 234
Try: 14 More particles accepted: 250
Not enough particles accepted: 250
# Created particles: 22432
# particles better than stop: 0
Min distance of current 

#### Fit 115 model to 14 pM

In [20]:
ind = np.where(smc_14pM[0][:,1,-1] == 5)[0]
particles = np.vstack((smc_14pM[0][ind,:,-1],np.zeros((1000 - ind.shape[0],20))))
particles[:,0] = 1./1000.
sims = np.dstack((smc_14pM[1][:,:,ind],np.zeros((smc_14pM[1].shape[0],smc_14pM[1].shape[1],1000 - ind.shape[0]))))
dd = np.zeros((1000 - ind.shape[0],6))
dd[:,5] = np.linspace(5,10,1000 - ind.shape[0])
dists = np.vstack((smc_14pM[2][ind,:,-1],dd))
perts = np.dstack((smc_14pM[3][:,:,ind],np.zeros((smc_14pM[3].shape[0],smc_14pM[3].shape[1],1000 - ind.shape[0]))))
start_population = [particles,sims,dists,perts]

In [21]:
alpha = 33.15
threshold = 1.
save = [True,path+'14pM/','smc_14pM_115_']
smc = smc_abc(Data14pM,Mock14pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'14pM/smc_14pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.587
Mean start distance: 2.097
Max start distance: 10.0
STD of start distance: 2.77
Mean of burst, tau and T: [  5.738   0.704  20.898]
STD of burst, tau and T: [  4.559   0.45   12.322]
Iteration: 1
Threshold distance 0.735
Try: 0 First particles accepted: 17
Try: 1 More particles accepted: 26
Try: 2 More particles accepted: 43
Try: 3 More particles accepted: 57
Try: 4 More particles accepted: 64
Try: 5 More particles accepted: 68
Try: 6 More particles accepted: 75
Try: 7 More particles accepted: 88
Try: 8 More particles accepted: 94
Try: 9 More particles accepted: 106
Try: 10 More particles accepted: 116
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 123
Try: 12 More particles accepted: 130
Try: 13 More particles accepted: 132
Try: 14 More particles accepted: 138
Not enough particles accepted: 138
# Created particles: 10861
# particles better than stop: 0
Min distance of current pop

### 20 pM, 2nd

In [22]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_20pM,par_cand_20pM = particle_distance_lut(Data20pM,Mock20pM,paths)
np.save(path + '20pM/dist_cand_20pM.npy',dist_cand_20pM)
np.save(path+'20pM/par_cand_20pM.npy',par_cand_20pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_20pM,n_particles)
np.save(path+'20pM/start_20pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 142.96 min
Duration: 2.48 min


In [23]:
start_population = np.load(path+'20pM/start_20pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [24]:
alpha = 33.15
threshold = 0.6
save = [True,path+'20pM/','smc_20pM_']
smc = smc_abc(Data20pM,Mock20pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'20pM/smc_20pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.506
Mean start distance: 0.971
Max start distance: 1.098
STD of start distance: 0.1
Mean of burst, tau and T: [ 12.311   2.493  21.765]
STD of burst, tau and T: [  6.697   1.547  11.185]
Iteration: 1
Threshold distance 0.887
Try: 0 First particles accepted: 3
Try: 1 More particles accepted: 16
Try: 2 More particles accepted: 33
Try: 3 More particles accepted: 55
Try: 4 More particles accepted: 72
Try: 5 More particles accepted: 105
Try: 6 More particles accepted: 127
Try: 7 More particles accepted: 148
Try: 8 More particles accepted: 173
Try: 9 More particles accepted: 198
Try: 10 More particles accepted: 221
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 242
Try: 12 More particles accepted: 258
Try: 13 More particles accepted: 279
Try: 14 More particles accepted: 299
Not enough particles accepted: 299
# Created particles: 22070
# particles better than stop: 0
Min distance of current 

#### Fit 115 model to 20 pM data

In [22]:
alpha = 33.15
threshold = 1.
ind = np.where(smc_20pM[0][:,1,-1] == 5)[0]
n_new = 1000 - ind.shape[0]
max_dist = 1.

# not enough 115 particles, fill up the start population by a prior iteration of the SMC algorithm
new_particles = create_new_particles_below_max_dist(Data20pM, Mock20pM, smc_20pM[0][ind,:,-1], max_dist, 
                                                    n_new, alpha, threshold,)

Try: 0 First particles accepted: 442
Try: 1 More particles accepted: 620
Try: 2 More particles accepted: 732
Try: 3 More particles accepted: 808
Try: 4 More particles accepted: 855
Try: 5 More particles accepted: 882
Try: 6 More particles accepted: 906
Try: 7 More particles accepted: 922
Try: 8 More particles accepted: 938
Try: 9 More particles accepted: 950
Try: 10 More particles accepted: 958
Try: 11 More particles accepted: 963
Try: 12 More particles accepted: 968
Try: 13 No particles accepted
Try: 14 More particles accepted: 970
Try: 15 More particles accepted: 972
Try: 16 More particles accepted: 974
Try: 17 More particles accepted: 976
Try: 18 No particles accepted
Try: 19 More particles accepted: 977
Not enough particles accepted: 977
# Created particles: 2802


In [23]:
dists = np.vstack((smc_20pM[2][ind,:,-1],new_particles[2]))
ind_sort = np.argsort(dists[:,-1])
dists = dists[ind_sort]
particles = np.vstack((smc_20pM[0][ind,:,-1],new_particles[0]))
particles = particles[ind_sort]
sims = np.dstack((smc_20pM[1][:,:,ind],new_particles[1]))
sims = sims[:,:,ind_sort]
perts = np.dstack((smc_20pM[3][:,:,ind],new_particles[3]))
perts = perts[:,:,ind_sort]
start_population = [particles,sims,dists,perts]
start_population[0][:,0] = 1./1000.

In [24]:
save = [True,path+'20pM/','smc_20pM_115_']
smc = smc_abc(Data20pM,Mock20pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'20pM/smc_20pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.573
Mean start distance: 0.86
Max start distance: 1.0
STD of start distance: 0.094
Mean of burst, tau and T: [ 10.779   0.833  17.886]
STD of burst, tau and T: [ 4.807  0.336  7.816]
Iteration: 1
Threshold distance 0.775
Try: 0 First particles accepted: 84
Try: 1 More particles accepted: 133
Try: 2 More particles accepted: 183
Try: 3 More particles accepted: 217
Try: 4 More particles accepted: 251
Try: 5 More particles accepted: 285
Try: 6 More particles accepted: 307
Try: 7 More particles accepted: 330
Try: 8 More particles accepted: 356
Try: 9 More particles accepted: 376
Try: 10 More particles accepted: 408
Try: 11 More particles accepted: 436
Try: 12 More particles accepted: 453
Try: 13 More particles accepted: 472
Try: 14 More particles accepted: 493
Try: 15 More particles accepted: 515
Try: 16 More particles accepted: 538
Try: 17 More particles accepted: 557
Try: 18 More particles accepted: 573
Try: 19 More particles accepted:

### 100 pM, 2nd

In [25]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_100pM,par_cand_100pM = particle_distance_lut(Data100pM,Mock100pM,paths)
np.save(path + '100pM/dist_cand_100pM.npy',dist_cand_100pM)
np.save(path+'100pM/par_cand_100pM.npy',par_cand_100pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_100pM,n_particles)
np.save(path+'100pM/start_100pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 114.12 min
Duration: 1.84 min


In [ ]:
start_population = np.load(path+'100pM/start_100pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [ ]:
alpha = 23.3
threshold = 0.6
save = [True,path+'100pM/','smc_100pM_']
smc = smc_abc(Data100pM_2,Mock100pM_2,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'100pM/smc_100pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.746
Mean start distance: 1.172
Max start distance: 1.289
STD of start distance: 0.093
Mean of burst, tau and T: [ 17.658   4.128  25.478]
STD of burst, tau and T: [  9.62    3.472  13.08 ]
Iteration: 1
Threshold distance 1.095
Try: 0 No particles accepted
Try: 1 No particles accepted
Try: 2 No particles accepted
Try: 3 No particles accepted
Try: 4 No particles accepted
Try: 5 No particles accepted
Try: 6 No particles accepted
Try: 7 No particles accepted
Try: 8 No particles accepted
Try: 9 No particles accepted
Try: 10 First particles accepted: 2
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 10
Try: 12 More particles accepted: 17
Try: 13 More particles accepted: 26
Try: 14 More particles accepted: 35
Not enough particles accepted: 35
# Created particles: 23945
# particles better than stop: 0
Min distance of current population: 0.746
Mean distance of current population: 1.146
Max dist

#### Fit 115 model to 100 pM

In [28]:
# load 100 pM fitting results
smc_100pM = np.load(path+'100pM/smc_100pM_2000_lut.npy')

In [29]:
alpha = 23.3
threshold = 1.
ind = np.where(smc_100pM[0][:,1,-1] == 5)[0]
n_new = 1000 - ind.shape[0]
max_dist = 1.5

new_particles = create_new_particles_below_max_dist(Data100pM, Mock100pM, smc_100pM[0][ind,:,-1], max_dist, 
                                                    n_new, alpha, threshold,)

Try: 0 First particles accepted: 681
Try: 1 More particles accepted: 827
Try: 2 More particles accepted: 897
Try: 3 More particles accepted: 928
Try: 4 More particles accepted: 951
Try: 5 More particles accepted: 963
Try: 6 More particles accepted: 973
Enough particles accepted: 973
# Created particles: 1550


In [30]:
dists = np.vstack((smc_100pM[2][ind,:,-1],new_particles[2]))
ind_sort = np.argsort(dists[:,-1])
dists = dists[ind_sort]
particles = np.vstack((smc_100pM[0][ind,:,-1],new_particles[0]))
particles = particles[ind_sort]
sims = np.dstack((smc_100pM[1][:,:,ind],new_particles[1]))
sims = sims[:,:,ind_sort]
perts = np.dstack((smc_100pM[3][:,:,ind],new_particles[3]))
perts = perts[:,:,ind_sort]
start_population = [particles,sims,dists,perts]
start_population[0][:,0] = 1./1000.
np.save(path+'100pM/start_100pM_1000_115.npy',start_population)

In [31]:
save = [True,path+'100pM/','smc_100pM_115_']
smc = smc_abc(Data100pM,Mock100pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'100pM/smc_100pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.816
Mean start distance: 1.279
Max start distance: 1.496
STD of start distance: 0.139
Mean of burst, tau and T: [ 34.447   9.993  33.34 ]
STD of burst, tau and T: [ 21.497   7.958  22.507]
Iteration: 1
Threshold distance 1.157
Try: 0 First particles accepted: 123
Try: 1 More particles accepted: 175
Try: 2 More particles accepted: 243
Try: 3 More particles accepted: 291
Try: 4 More particles accepted: 340
Try: 5 More particles accepted: 381
Try: 6 More particles accepted: 426
Try: 7 More particles accepted: 459
Try: 8 More particles accepted: 498
Try: 9 More particles accepted: 526
Try: 10 More particles accepted: 551
Try: 11 More particles accepted: 577
Try: 12 More particles accepted: 598
Try: 13 More particles accepted: 616
Try: 14 More particles accepted: 639
Try: 15 More particles accepted: 648
Try: 16 More particles accepted: 657
Try: 17 More particles accepted: 668
Try: 18 More particles accepted: 681
Try: 19 More particles ac

### 1000 pM, 2nd

In [8]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_1000pM,par_cand_1000pM = particle_distance_lut(Data1000pM,Mock1000pM,paths)
np.save(path + '1000pM/dist_cand_1000pM.npy',dist_cand_1000pM)
np.save(path+'1000pM/par_cand_1000pM.npy',par_cand_1000pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_1000pM,n_particles)
np.save(path+'1000pM/start_1000pM_2000_corrected.npy',start_population)

File: 1 / 35
File: 2 / 35
File: 3 / 35
File: 4 / 35
File: 5 / 35
File: 6 / 35
File: 7 / 35
File: 8 / 35
File: 9 / 35
File: 10 / 35
File: 11 / 35
File: 12 / 35
File: 13 / 35
File: 14 / 35
File: 15 / 35
File: 16 / 35
File: 17 / 35
File: 18 / 35
File: 19 / 35
File: 20 / 35
File: 21 / 35
File: 22 / 35
File: 23 / 35
File: 24 / 35
File: 25 / 35
File: 26 / 35
File: 27 / 35
File: 28 / 35
File: 29 / 35
File: 30 / 35
File: 31 / 35
File: 32 / 35
File: 33 / 35
File: 34 / 35
File: 35 / 35
Duration: 100.5 min
Duration: 1.74 min


In [ ]:
start_population = np.load(path+'1000pM/start_1000pM_2000_corrected.npy')
start_population[0][:,0] = 1./n_particles

In [ ]:
alpha = 23.3
threshold = 0.6
save = [True,path+'1000pM/','smc_1000pM_']
smc = smc_abc(Data1000pM,Mock1000pM,start_population,iterations,stop,alpha = alpha,save = save)
np.save(path+'1000pM/smc_1000pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.954
Mean start distance: 1.471
Max start distance: 1.612
STD of start distance: 0.118
Mean of burst, tau and T: [ 19.281   4.838  15.694]
STD of burst, tau and T: [ 11.221   4.849   9.195]
Iteration: 1
Threshold distance 1.381


<string>:748: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
<string>:749: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Try: 0 No particles accepted
Try: 1 No particles accepted
Try: 2 First particles accepted: 1
Try: 3 More particles accepted: 3


#### Fit 115 model to 1000 pM

In [9]:
# load 1000 pM data
smc_1000pM = np.load(path+'1000pM/smc_1000pM_2000_lut.npy')

In [10]:
alpha = 23.3
threshold = 1.
ind = np.where(smc_1000pM[0][:,1,-1] == 5)[0]

In [22]:
n_new = 1000 - ind.shape[0]
max_dist = 1.5
new_particles = create_new_particles_below_max_dist(Data1000pM, Mock1000pM, smc_1000pM[0][ind,:,-1], max_dist, 
                                                    n_new, alpha, threshold,)

Try: 0 First particles accepted: 159
Try: 1 More particles accepted: 278
Try: 2 More particles accepted: 385
Try: 3 More particles accepted: 468
Try: 4 More particles accepted: 538
Try: 5 More particles accepted: 597
Try: 6 More particles accepted: 636
Try: 7 More particles accepted: 683
Try: 8 More particles accepted: 716
Try: 9 More particles accepted: 736
Try: 10 More particles accepted: 757
Try: 11 More particles accepted: 780
Try: 12 More particles accepted: 808
Try: 13 More particles accepted: 832
Try: 14 More particles accepted: 853
Try: 15 More particles accepted: 871
Try: 16 More particles accepted: 887
Try: 17 More particles accepted: 899
Try: 18 More particles accepted: 905
Try: 19 More particles accepted: 917
Not enough particles accepted: 917
# Created particles: 6792


In [30]:
dists = np.vstack((smc_1000pM[2][ind,:,-1],new_particles[2]))
ind_sort = np.argsort(dists[:,-1])
dists = dists[ind_sort]
particles = np.vstack((smc_1000pM[0][ind,:,-1],new_particles[0]))
particles = particles[ind_sort]
sims = np.dstack((smc_1000pM[1][:,:,ind],new_particles[1]))
sims = sims[:,:,ind_sort]
perts = np.dstack((smc_1000pM[3][:,:,ind],new_particles[3]))
perts = perts[:,:,ind_sort]
start_population = [particles,sims,dists,perts]
start_population[0][:,0] = 1./1000.
np.save(path+'1000pM/start_1000pM_115.npy',start_population)

In [34]:
start_population = np.load(path+'1000pM/start_1000pM_115.npy')

In [38]:
alpha = 23.15
threshold = 2.0
save = [True,path+'1000pM/','smc_1000pM_115_']
smc = smc_abc(Data1000pM,Mock1000pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'1000pM/smc_1000pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 1.118
Mean start distance: 1.427
Max start distance: 1.508
STD of start distance: 0.063
Mean of burst, tau and T: [ 71.34   27.184  22.62 ]
STD of burst, tau and T: [ 24.436  13.577   7.66 ]
Iteration: 1
Threshold distance 1.379
Try: 0 First particles accepted: 38
Try: 1 More particles accepted: 56
Try: 2 More particles accepted: 77
Try: 3 More particles accepted: 93
Try: 4 More particles accepted: 117
Try: 5 More particles accepted: 142
Try: 6 More particles accepted: 169
Try: 7 More particles accepted: 195
Try: 8 More particles accepted: 222
Try: 9 More particles accepted: 240
Try: 10 More particles accepted: 272
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 285
Try: 12 More particles accepted: 307
Try: 13 More particles accepted: 329
Try: 14 More particles accepted: 344
Not enough particles accepted: 344
# Created particles: 9458
# particles better than stop: 0
Min distance of curre

# Inhibitors

In [50]:
# load experimental data
Data20pM_NaBu = np.load('Data20pM_NaBu.npy')
Data20pM_C646 = np.load('Data20pM_C646.npy')
Data20pM_DMSO = np.load('Data20pM_DMSO.npy')

Mock20pM_NaBu = np.load('Mock20pM_NaBu.npy')
Mock20pM_C646 = np.load('Mock20pM_C646.npy')
Mock20pM_DMSO = np.load('Mock20pM_DMSO.npy')

path = '/Users/stephan/Desktop/gene_transcription_SMC_ABC/fits/inhibitors/'

## 20 pM estrogen + sodium butyrate

HDAC inhibitor

In [54]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_20pM_NaBu,par_cand_20pM_NaBu = particle_distance_lut(Data20pM_NaBu,Mock20pM_NaBu,paths)
np.save(path + '20pM_NaBu/dist_cand_20pM_NaBu.npy',dist_cand_20pM_NaBu)
np.save(path+'20pM_NaBu/par_cand_20pM_NaBu.npy',par_cand_20pM_NaBu)

File: 1 / 2
File: 2 / 2
Duration: 0.35 min


IOError: [Errno 2] No such file or directory: '/Users/stephan/Desktop/gene_transcription_SMC_ABC/fits/20pM_NaBu/dist_cand_20pM_NaBu.npy'

In [ ]:
files = corrected_filelist(temp_20pM)
start_population = best_start_particles(files,dist_cand_20pM_NaBu,n_particles)
np.save(path+'20pM_NaBu/start_2000_20pM_NaBu.npy',start_population)

In [ ]:
start_population = np.load(path+'20pM_NaBu/start_2000_20pM_NaBu.npy')
start_population[0][:,0] = 1./n_particles

In [ ]:
alpha = 23.3
threshold = 0.6
save = [True,path+'20pM_NaBu/','smc_20pM_NaBu_']
smc = smc_abc(Data20pM_NaBu,Mock20pM_NaBu,start_population,iterations,stop,alpha = alpha,save = save)
np.save(path+'20pM_NaBu/smc_20pM_NaBu_2000_lut.npy',smc)

### Fit 115 model to NaBu 

In [15]:
# generate new particles to fill start population
alpha = 23.3
threshold = 1.
ind = np.where(smc_NaBu[0][:,1,-1] == 5)[0]
n_new = 1000 - ind.shape[0]
max_dist = 1.4
new_particles = create_new_particles_below_max_dist(Data20pM_NaBu, Mock20pM_NaBu, smc_NaBu[0][ind,:,-1], max_dist, 
                                                    n_new, alpha, threshold,)

Try: 0 First particles accepted: 6
Try: 1 More particles accepted: 13
Try: 2 More particles accepted: 15
Try: 3 More particles accepted: 22
Try: 4 More particles accepted: 24
Try: 5 More particles accepted: 30
Try: 6 More particles accepted: 32
Try: 7 More particles accepted: 33
Try: 8 More particles accepted: 36
Try: 9 More particles accepted: 39
Try: 10 More particles accepted: 44
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 51
Try: 12 More particles accepted: 60
Try: 13 More particles accepted: 64
Try: 14 More particles accepted: 69
Not enough particles accepted: 69
# Created particles: 1991


In [16]:
# create start population with existing and newly created particles
dists = np.vstack((smc_NaBu[2][ind,:,-1],new_particles[2]))
ind_sort = np.argsort(dists[:,-1])
dists = dists[ind_sort]
particles = np.vstack((smc_NaBu[0][ind,:,-1],new_particles[0]))
particles = particles[ind_sort]
sims = np.dstack((smc_NaBu[1][:,:,ind],new_particles[1]))
sims = sims[:,:,ind_sort]
perts = np.dstack((smc_NaBu[3][:,:,ind],new_particles[3]))
perts = perts[:,:,ind_sort]
start_population = [particles,sims,dists,perts]
start_population[0][:,0] = 1./1000.
np.save(path+'20pM_NaBu/start_NaBu_1000_20pM_NaBu.npy',start_population)

In [24]:
save = [True,path+'20pM_NaBu/','smc_NaBu_115_']
smc = smc_abc(Data20pM_NaBu,Mock20pM_NaBu,start_population,iterations,stop,threshold=threshold,
              alpha = alpha,save = save)
np.save(path+'20pM_NaBu/smc_NaBu_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 1.104
Mean start distance: 1.377
Max start distance: 1.441
STD of start distance: 0.051
Mean of burst, tau and T: [  8.98  11.16  53.15]
STD of burst, tau and T: [  6.879  12.184  47.01 ]
Iteration: 1
Threshold distance 1.343
Try: 0 First particles accepted: 32
Try: 1 More particles accepted: 50
Try: 2 More particles accepted: 77
Try: 3 More particles accepted: 100
Try: 4 More particles accepted: 131
Try: 5 More particles accepted: 165
Try: 6 More particles accepted: 183
Try: 7 More particles accepted: 203
Try: 8 More particles accepted: 223
Try: 9 More particles accepted: 252
Try: 10 More particles accepted: 274
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 302
Try: 12 More particles accepted: 324
Try: 13 More particles accepted: 344
Try: 14 More particles accepted: 368
Not enough particles accepted: 368
# Created particles: 9340
# particles better than stop: 0
Min distance of current

## 20 pM estrogen + DMSO

Control condition

In [ ]:
# compare data to blind shooting candidates
temp_20pM_DMSO= 'H:/stephan/temp/'
paths = [cand_path,temp_20pM_DMSO]
dist_cand_20pM_DMSO,par_cand_20pM_DMSO = particle_distance_lut(Data20pM_DMSO,Mock20pM_DMSO,paths)

In [ ]:
np.save(path + '20pM_DMSO/dist_cand_20pM_DMSO.npy',dist_cand_20pM_DMSO)
np.save(path+'20pM_DMSO/par_cand_20pM_DMSO.npy',par_cand_20pM_DMSO)

files = corrected_filelist(temp_20pM_DMSO)
start_population = best_start_particles(files,dist_cand_20pM_DMSO,n_particles)
np.save(path+'20pM_DMSO/start_2000_20pM_DMSO.npy',start_population)

In [ ]:
start_population = np.load(path+'20pM_DMSO/start_2000_20pM_DMSO.npy')
start_population[0][:,0] = 1./n_particles

In [ ]:
alpha = 23.3
threshold = 0.6
save = [True,path+'20pM_DMSO/','smc_20pM_DMSO_']
smc = smc_abc(Data20pM_DMSO,Mock20pM_DMSO,start_population,iterations,stop,alpha = alpha,save = save)
np.save(path+'20pM_DMSO/smc_20pM_DMSO_2000_lut.npy',smc)

### Fit 115 model to DMSO

In [25]:
alpha = 23.3
threshold = 1.
ind = np.where(smc_DMSO[0][:,1,-1] == 5)[0]

n_new = 1000 - ind.shape[0]
max_dist = 1.
new_particles = create_new_particles_below_max_dist(Data20pM_DMSO, Mock20pM_DMSO, smc_DMSO[0][ind,:,-1], max_dist, 
                                                    n_new, alpha, threshold,)

Try: 0 First particles accepted: 145
Try: 1 More particles accepted: 237
Try: 2 More particles accepted: 320
Try: 3 More particles accepted: 375
Try: 4 More particles accepted: 434
Try: 5 More particles accepted: 489
Try: 6 More particles accepted: 532
Try: 7 More particles accepted: 565
Try: 8 More particles accepted: 605
Try: 9 More particles accepted: 646
Try: 10 More particles accepted: 678
Try: 11 More particles accepted: 704
Try: 12 More particles accepted: 724
Try: 13 More particles accepted: 750
Try: 14 More particles accepted: 767
Try: 15 More particles accepted: 780
Try: 16 More particles accepted: 792
Try: 17 More particles accepted: 801
Try: 18 More particles accepted: 809
Try: 19 More particles accepted: 820
Not enough particles accepted: 820
# Created particles: 7027


In [26]:
dists = np.vstack((smc_DMSO[2][ind,:,-1],new_particles[2]))
ind_sort = np.argsort(dists[:,-1])
dists = dists[ind_sort]
particles = np.vstack((smc_DMSO[0][ind,:,-1],new_particles[0]))
particles = particles[ind_sort]
sims = np.dstack((smc_DMSO[1][:,:,ind],new_particles[1]))
sims = sims[:,:,ind_sort]
perts = np.dstack((smc_DMSO[3][:,:,ind],new_particles[3]))
perts = perts[:,:,ind_sort]
start_population = [particles,sims,dists,perts]
start_population[0][:,0] = 1./1000.
np.save(path+'20pM_DMSO/start_DMSO_1000_20pM_NaBu.npy',start_population)

In [27]:
save = [True,path+'20pM_DMSO/','smc_DMSO_115_']
smc = smc_abc(Data20pM_DMSO,Mock20pM_DMSO,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'20pM_DMSO/smc_DMSO_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.67
Mean start distance: 0.921
Max start distance: 1.01
STD of start distance: 0.071
Mean of burst, tau and T: [ 15.297   1.316  19.64 ]
STD of burst, tau and T: [ 4.871  0.413  6.354]
Iteration: 1
Threshold distance 0.852
Try: 0 First particles accepted: 22
Try: 1 More particles accepted: 38
Try: 2 More particles accepted: 54
Try: 3 More particles accepted: 69
Try: 4 More particles accepted: 92
Try: 5 More particles accepted: 107
Try: 6 More particles accepted: 121
Try: 7 More particles accepted: 135
Try: 8 More particles accepted: 148
Try: 9 More particles accepted: 158
Try: 10 More particles accepted: 170
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 183
Try: 12 More particles accepted: 188
Try: 13 More particles accepted: 197
Try: 14 More particles accepted: 203
Not enough particles accepted: 203
# Created particles: 10318
# particles better than stop: 0
Min distance of current po

## 20 pM estrogen + C646
p300 (histone acetyl transferase) inhibitor

In [ ]:
# compare data to blind shooting candidates
temp_20pM_C646= 'H:/stephan/temp/'
paths = [cand_path,temp_20pM_C646]
dist_cand_20pM_C646,par_cand_20pM_C646 = particle_distance_lut(Data20pM_C646,Mock20pM_C646,paths)

In [ ]:
np.save(path + '20pM_C646/dist_cand_20pM_C646.npy',dist_cand_20pM_C646)
np.save(path+'20pM_C646/par_cand_20pM_C646.npy',par_cand_20pM_C646)

files = corrected_filelist(temp_20pM_C646)
start_population = best_start_particles(files,dist_cand_20pM_C646,n_particles)
np.save(path+'20pM_C646/start_2000_20pM_C646.npy',start_population)

In [ ]:
start_population = np.load(path+'20pM_C646/start_2000_20pM_C646.npy')
start_population[0][:,0] = 1./n_particles

In [ ]:
alpha = 23.3
threshold = 0.6
save = [True,path+'20pM_C646/','smc_20pM_C646_']
smc = smc_abc(Data20pM_C646,Mock20pM_C646,start_population,iterations,stop,alpha = alpha,save = save)
np.save(path+'20pM_C646/smc_20pM_C646_2000_lut.npy',smc)

### Fit 115 model to C646

In [30]:
alpha = 23.3
threshold = 1.
ind = np.where(smc_C646[0][:,1,-1] == 5)[0]

n_new = 1000 - ind.shape[0]
max_dist = 1.
new_particles = create_new_particles_below_max_dist(Data20pM_C646, Mock20pM_C646, smc_C646[0][ind,:,-1], max_dist, 
                                                    n_new, alpha, threshold,)

Try: 0 First particles accepted: 7
Try: 1 More particles accepted: 12
Try: 2 More particles accepted: 17
Try: 3 More particles accepted: 20
Try: 4 More particles accepted: 24
Try: 5 More particles accepted: 27
Try: 6 More particles accepted: 35
Try: 7 More particles accepted: 36
Try: 8 More particles accepted: 43
Try: 9 More particles accepted: 48
Try: 10 More particles accepted: 55
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 60
Try: 12 More particles accepted: 70
Try: 13 More particles accepted: 80
Try: 14 More particles accepted: 87
Not enough particles accepted: 87
# Created particles: 14346


In [31]:
dists = np.vstack((smc_C646[2][ind,:,-1],new_particles[2]))
ind_sort = np.argsort(dists[:,-1])
dists = dists[ind_sort]
particles = np.vstack((smc_C646[0][ind,:,-1],new_particles[0]))
particles = particles[ind_sort]
sims = np.dstack((smc_C646[1][:,:,ind],new_particles[1]))
sims = sims[:,:,ind_sort]
perts = np.dstack((smc_C646[3][:,:,ind],new_particles[3]))
perts = perts[:,:,ind_sort]
start_population = [particles,sims,dists,perts]
start_population[0][:,0] = 1./1000.

In [9]:
start_population = np.load(path+'20pM_C646/start_1000_20pM_C646_115.npy')

In [15]:
alpha = 23.3
threshold = 1.
save = [True,path+'20pM_C646/','smc_C646_115_']
smc = smc_abc(Data20pM_C646,Mock20pM_C646,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'20pM_C646/smc_C646_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.871
Mean start distance: 1.074
Max start distance: 1.142
STD of start distance: 0.051
Mean of burst, tau and T: [ 12.225   0.584  10.413]
STD of burst, tau and T: [ 3.858  0.124  3.32 ]
Iteration: 1
Threshold distance 1.033
Try: 0 First particles accepted: 7
Try: 1 More particles accepted: 16
Try: 2 More particles accepted: 29
Try: 3 More particles accepted: 33
Try: 4 More particles accepted: 41
Try: 5 More particles accepted: 50
Try: 6 More particles accepted: 58
Try: 7 More particles accepted: 67
Try: 8 More particles accepted: 74
Try: 9 More particles accepted: 84
Try: 10 More particles accepted: 91
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 104
Try: 12 More particles accepted: 113
Try: 13 More particles accepted: 119
Try: 14 More particles accepted: 123
Not enough particles accepted: 123
# Created particles: 11114
# particles better than stop: 0
Min distance of current populat